In [1]:
import torch
import torchvision
import torchaudio

In [17]:
 torch.cuda.empty_cache()

In [3]:
torch.__version__

'2.1.1+cu121'

In [4]:
#check for GPU
torch.cuda.is_available()
# device = torch.device('cuda')
# device

True

In [5]:
import numpy as np
import json
import random
import time
from transformers import BertTokenizer

In [6]:
map_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
                 'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
                 'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}

In [2]:
home=%pwd
filename = home + '/data/TRAIN_407.json'

In [7]:
from utils import load_data, input_format, position_ids_compute, tokenize
from bert_format import undersample, format_time, flat_accuracy

In [10]:
data = load_data(filename, map_relations)

Loading data: /home/kate/LREC/data/TRAIN_407.json
407 dialogs, 21822 edus, 26299 relations, 194 backward relations
4787 edus have multiple parents


In [11]:
#split out a certain portion of validation data (a function of length?)
train_data = data[40:]
valid_data = data[:40]

In [12]:
input_text_train, labels_train, raw_train = input_format(train_data, 10)

23122 relations
175919 candidates
152797 non attached


In [13]:
input_text_valid, labels_valid, raw_valid = input_format(valid_data, 10)

2592 relations
19920 candidates
17328 non attached


In [11]:
#load tokenizer and token ids
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', use_fast=True)

In [12]:
put = ['1','0']
colors = ['r', 'b', 'g', 'o', 'y', 'p']
listx = ['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n']
listy = ['0', '1', '2', '3', '4', '5', '6', '7', '8']
listz = ['a', 'e', 'i', 'o', 'u', 'p', 'q', 'r', 'x', 'y', 'z']

In [13]:
coord_tokens = [''.join([s, t, i, j, k]) for s in put
                for t in colors
                for i in listx
                for j in listy
                for k in listz]

In [14]:
tokenizer.add_tokens(coord_tokens)

13068

In [15]:
len(tokenizer)

42064

In [16]:
device = torch.device('cuda')

In [20]:
input_ids_train, attention_masks_train, token_type_ids_train = tokenize(input_text_train, tokenizer, device)

In [21]:
input_ids_valid, attention_masks_valid, token_type_ids_valid = tokenize(input_text_valid, tokenizer, device)

compute position ids

In [22]:
position_ids_train = position_ids_compute(tokenizer, input_ids_train, raw_train, labels_train)

In [23]:
position_ids_valid = position_ids_compute(tokenizer, input_ids_valid, raw_valid, labels_valid)

In [24]:
position_ids_train = torch.tensor(position_ids_train)

In [48]:
position_ids_valid = torch.tensor(position_ids_valid)

Undersample <br>
For Bertlinear we use the undersample function because...

In [26]:
from bert_format import undersample

In [27]:
#all cands
print(len(labels_train))
#unattached cands
print(sum([1 for i in labels_train if i[3] == 0]))
#attached cands
print(sum([1 for i in labels_train if i[3] == 1]))

175919
152797
23122


In [28]:
labels_attach_train = [l[3] for l in labels_train]

In [29]:
labels_attach_valid = [l[3] for l in labels_valid]

In [30]:
labels_train = torch.tensor(labels_train)
labels_valid = torch.tensor(labels_valid)
labels_attach_train = torch.tensor(labels_attach_train)
labels_attach_valid = torch.tensor(labels_attach_valid)

In [31]:
labels_train, labels_attach_train, input_ids_train, attention_masks_train, token_type_ids_train, position_ids_train = undersample(68576, labels_train, labels_attach_train, input_ids_train, attention_masks_train, token_type_ids_train, position_ids_train)

gather metadata from labels

In [32]:
#make meta data -- lists of distances -- ADD INFO FOR RESULT
meta_data_train = []
for i in range(len(labels_train)):
  lbs = labels_train[i].tolist()
  # meta_data_train.append([lbs[2], lbs[2]-lbs[1], lbs[5]])
  # meta_data_train.append([lbs[2], lbs[2]-lbs[1], lbs[5], lbs[6]])
  meta_data_train.append([lbs[2], lbs[2]-lbs[1]])

meta_data_valid = []
for i in range(len(labels_valid)):
  lbs = labels_valid[i].tolist()
  # meta_data_valid.append([lbs[2], lbs[2]-lbs[1], lbs[5]])
  # meta_data_valid.append([lbs[2], lbs[2]-lbs[1], lbs[5], lbs[6]])
  meta_data_valid.append([lbs[2], lbs[2]-lbs[1]])

create batches

In [24]:
#create metadata batches
def get_batches(len_data, batch_size):
    indices = [i for i in range(len_data)]
    batches = []
    for i in range(len_data // batch_size + bool(len_data) % batch_size):
        batches.append(indices[i * batch_size:(i + 1) * batch_size])
    return batches

In [34]:
train_batches = get_batches(len(meta_data_train), 32)

In [35]:
valid_batches = get_batches(len(meta_data_valid), 32)

In [36]:
meta_train_batches = []
for ba in train_batches:
  meta_train_batches.append([meta_data_train[b] for b in ba])

In [37]:
meta_valid_batches = []
for ba in valid_batches:
  meta_valid_batches.append([meta_data_valid[b] for b in ba])

In [23]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from transformers import AdamW, BertForSequenceClassification

In [39]:
train_dataset = TensorDataset(input_ids_train, attention_masks_train, token_type_ids_train, position_ids_train, labels_attach_train)

In [47]:
input_ids_train.size(), attention_masks_train.size(), position_ids_train.size(), labels_attach_train.size()

(torch.Size([91698, 113]),
 torch.Size([91698, 113]),
 torch.Size([91698, 113]),
 torch.Size([91698]))

In [49]:
input_ids_valid.size(), attention_masks_valid.size(), position_ids_valid.size(), labels_attach_valid.size()

(torch.Size([19920, 71]),
 torch.Size([19920, 71]),
 torch.Size([19920, 71]),
 torch.Size([19920]))

In [50]:
val_dataset = TensorDataset(input_ids_valid, attention_masks_valid, token_type_ids_valid, position_ids_valid, labels_attach_valid)

In [51]:
train_dataloader = DataLoader(
            train_dataset,
            sampler = SequentialSampler(train_dataset),
            batch_size = 32
        )

In [52]:
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = 32
        )

Load finetuned from first step

In [37]:
model_path = home + '/models/finetune_d10.pth'

In [38]:
embedder = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    output_attentions = False,
    output_hidden_states = True, attention_probs_dropout_prob=0, hidden_dropout_prob=0
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
#!!resize embedder to account for new embeddings!
embedder.resize_token_embeddings(len(tokenizer))

Embedding(42064, 768)

In [40]:
checkpoint = torch.load(model_path, map_location=device)
embedder.load_state_dict(checkpoint['model_state_dict'])
embedder.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42064, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

load NN

In [32]:
import random
import pickle
from torch import nn

In [33]:
#fix neural net here
#hidden_size = 774
#hidden_size = 772 for just incoming rels

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.linear = nn.Sequential(
            nn.Dropout(p=0.3),
            # nn.Linear(params.hidden_size, params.hidden_size_1),
            nn.Linear(770, 2000),
            nn.Dropout(p=0.3),
            nn.Tanh(),
            nn.Linear(2000, 1))


    def forward(self, x):
        logits = self.linear(x)
        return logits

In [34]:
linear = NeuralNetwork().to(device)

In [60]:
linear.train()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=linear.parameters(), lr=0.0002)

In [61]:
linear_model_path = home + '/models/'
save_linear_name =  'linear_d10_pass1.pth'

train the model

In [62]:
for epoch in range(15):
    loss_sum_train = 0
    linear.train()
    for e, batch in enumerate(train_dataloader):
      if e in [0, len(train_dataloader)-1]:
        print("epoch ", epoch)
        print("batch no ", e)
      output = embedder(batch[0].to(device),
                        token_type_ids = batch[2].to(device),
                        attention_mask = batch[1].to(device),
                        position_ids = batch[3].to(device),
                        labels = batch[4].to(device),
                        return_dict=True)
      #concat each candidate embedding with metadata tensor
      #stack these
      H_embed = torch.stack([torch.cat((r[0], torch.tensor(meta_train_batches[e][i]).to(device)),0) for i, r in enumerate(output.hidden_states[-1])])
      H_embed = H_embed.to(device)
      logits = linear(H_embed).unsqueeze(0)
      logits = logits.squeeze(-1)

      #print(batch[4])

      target = torch.tensor([[float(b) for b in batch[4]]]).to(device)
      #target = float(batch[4][None,:]).to(device)
      #target = torch.tensor([[float(lab[3]) for lab in labels[i]]]).to(device)
      #need to detach?
      # print(target.size())
      # print(logits.size())

      loss = criterion(input=logits, target=target)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      loss_sum_train += loss.item()

    # Calculate the average train loss over all of the batches.
    avg_train_loss = loss_sum_train / len(train_dataloader)
    print("avg train loss: ", avg_train_loss)


    print("going to eval")
    linear.eval()
    loss_sum_valid = 0

    for e, batch in enumerate(validation_dataloader):
      with torch.no_grad():
        output = embedder(batch[0].to(device),
                        token_type_ids = batch[2].to(device),
                        attention_mask = batch[1].to(device),
                        position_ids = batch[3].to(device),
                        labels = batch[4].to(device),
                        return_dict=True)

      H_embed = torch.stack([torch.cat((r[0], torch.tensor(meta_valid_batches[e][i]).to(device)),0) for i, r in enumerate(output.hidden_states[-1])])
      H_embed = H_embed.to(device)
      with torch.no_grad():
            logits = linear(H_embed).unsqueeze(0)

      logits = logits.squeeze(-1)

      target = torch.tensor([[float(b) for b in batch[4]]]).to(device)
      # target = batch[4].to(device)

      loss = criterion(input=logits, target=target)

      loss_sum_valid += loss.item()

    # Calculate the average loss over all of the batches.
    avg_val_loss = loss_sum_valid / len(validation_dataloader)
    print("avg val loss: ", avg_val_loss)

    print('--------------------------------------')

output_model = linear_model_path + save_linear_name

print('finished_training, saving to : ', output_model)

torch.save({
    'model_state_dict': linear.state_dict(),
}, output_model)

epoch  0
batch no  0
epoch  0
batch no  2865
avg train loss:  0.12048575012318774
going to eval
avg val loss:  0.17581124330237718
--------------------------------------
epoch  1
batch no  0
epoch  1
batch no  2865
avg train loss:  0.11419450670814782
going to eval
avg val loss:  0.17970858346525112
--------------------------------------
epoch  2
batch no  0
epoch  2
batch no  2865
avg train loss:  0.1126828385401317
going to eval
avg val loss:  0.18190164687300706
--------------------------------------
epoch  3
batch no  0
epoch  3
batch no  2865
avg train loss:  0.11136398847693862
going to eval
avg val loss:  0.18053217681678294
--------------------------------------
epoch  4
batch no  0
epoch  4
batch no  2865
avg train loss:  0.11054066676408929
going to eval
avg val loss:  0.1840287333616522
--------------------------------------
epoch  5
batch no  0
epoch  5
batch no  2865
avg train loss:  0.10965474318961216
going to eval
avg val loss:  0.18483605031368813
---------------------

Get scores on test

In [49]:
#load test data
# filename = home + '/data/DEV_32_bert.json'
filename = home + '/data/TEST_102_bert.json'
test_data = load_data(filename, map_relations)

Loading data: /home/kate/LREC/data/TEST_102_bert.json
102 dialogs, 5032 edus, 6041 relations, 56 backward relations
1081 edus have multiple parents


In [50]:
input_text_test, labels_test, raw_test = input_format(test_data, 10)

5886 relations
44710 candidates
38824 non attached


In [51]:
input_ids_test, attention_masks_test, token_type_ids_test = tokenize(input_text_test, tokenizer, device)

In [52]:
position_ids_test = position_ids_compute(tokenizer, input_ids_test, raw_test, labels_test)

In [53]:
position_ids_test = torch.tensor(position_ids_test)

In [54]:
labels_attach_test = [l[3] for l in labels_test]

In [55]:
meta_data_test = []
for i in range(len(labels_test)):
  lbs = labels_test[i]
  # meta_data_test.append([lbs[2], lbs[2]-lbs[1], lbs[5]])
  # meta_data_test.append([lbs[2], lbs[2]-lbs[1], lbs[5], lbs[6]])
  meta_data_test.append([lbs[2], lbs[2]-lbs[1]])

In [56]:
test_batches = get_batches(len(meta_data_test), 32)

In [57]:
meta_test_batches = []
for ba in test_batches:
  meta_test_batches.append([meta_data_test[b] for b in ba])

In [58]:
labels_test_batches = []
for ba in test_batches:
  labels_test_batches.append([labels_test[b] for b in ba])
# labels_test_batches = []
# for ba in test_batches:
#   labels_test_batches.append([labels_test[b] for b in ba])

In [59]:
test_dataset = TensorDataset(input_ids_test, attention_masks_test, token_type_ids_test, position_ids_test)

In [60]:
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = 32
        )

(if need to reload models...)

In [35]:
model_path = home + '/models/' +  'linear_d10_pass1.pth'
linear = NeuralNetwork().to(device)
checkpoint = torch.load(model_path, map_location='cuda')
linear.load_state_dict(checkpoint['model_state_dict'])
linear.to(device)

NeuralNetwork(
  (linear): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=770, out_features=2000, bias=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Tanh()
    (4): Linear(in_features=2000, out_features=1, bias=True)
  )
)

In [61]:
predictions = []

linear.eval()

for e, batch in enumerate(test_dataloader):
  with torch.no_grad():
    output = embedder(batch[0].to(device),
                    token_type_ids = batch[2].to(device),
                    attention_mask = batch[1].to(device),
                    position_ids = batch[3].to(device),
                    # labels = batch[4].to(device),
                    return_dict=True)

  H_embed = torch.stack([torch.cat((r[0], torch.tensor(meta_test_batches[e][i]).to(device)),0) for i, r in enumerate(output.hidden_states[-1])])
  H_embed = H_embed.to(device)
  with torch.no_grad():
        logits = linear(H_embed).unsqueeze(0)

  #print(logits)
  m = nn.Sigmoid()
  mod =(m(logits)).squeeze(-1).cpu().tolist()[0]
  #print(mod)
  xs = [i for i in range(len(mod)) if mod[i] > 0.81]  # 0.95
  #print(xs)
  # print(len(xs))
  #add highest in batch if nothing is above 81?? why???
  # if len(xs) == 0 : xs += [ np.argmax([float(p[0]) for p in logits[0].cpu()])]
  # print(xs)

  labels = labels_test_batches[e]
  for lab in range(len(labels)):
    if lab in xs:
      labels[lab].append(1)
    else:
      labels[lab].append(0)

  predictions.extend(labels)



In [62]:
len(predictions)

44710

In [63]:
predictions[:3]

[[0, 0, 1, 1, 0, 1], [0, 0, 2, 1, 8, 1], [0, 1, 2, 0, -1, 0]]

In [64]:
attach_predictions = [i[5] for i in predictions]
true_labels = [i[3] for i in predictions]

In [65]:
len(attach_predictions), len(true_labels)

(44710, 44710)

In [45]:
from sklearn.metrics import precision_recall_fscore_support

In [66]:
precision_recall_fscore_support(true_labels, attach_predictions, average='binary')

(0.8278381416260772, 0.7507645259938838, 0.7874198146828225, None)

save predictions<br>
list of lists with [dialogue index, x index, y index, true attach, true label, predicted attach]

In [67]:
# import pickle 
# with open(home + '/pickles/' + 'scores_linear_d10_test.pkl', 'wb') as f:
#     pickle.dump(predictions, f)

change output to a list of lists so it can be fed to multitask <br>
needs to be a list of lists, each list a game

In [4]:
import pickle
with open(home + '/pickles/' + 'scores_linear_d10_dev.pkl', 'rb') as f:
    predictions = pickle.load(f)

In [5]:
multitask_inputs = []
for i in range(32):
    inputs = [[e[1], e[2]] for e in predictions if e[0] == i and e[5]==1]
    multitask_inputs.append(inputs)

In [6]:
len(multitask_inputs[1])

35

In [7]:
with open(home + '/pickles/' + 'MT_input_d10_DEV.pkl', 'wb') as f:
    pickle.dump(multitask_inputs, f)